In [2]:
import os
import numpy as np
import reading 
from feature_extractor import feature_extractor
import pandas as pd
import matplotlib.pyplot as plt

Load dei Path, Con il sort giusto per far corrispondere i soggetti

In [3]:
paths_MD= reading.data_path("Diffusion_parameters_maps-20230215T134959Z-001","corrected_MD_image")
paths_masks=reading.data_path("Diffusion_space_segmentations-20230215T134839Z-001","Diffusion_space_segmentations-20230215T134839Z-001")


#print((os.path.basename(paths_masks[0]).split('_')[2]))

paths_MD.sort(key=lambda x: int(os.path.basename(x).split('_')[3]))
paths_masks.sort(key=lambda x: int(os.path.basename(x).split('_')[2]))


Feature extraction

In [4]:
region, mean, std=feature_extractor(paths_MD,paths_masks)


Reshape degli array per fare il Dataframe

In [5]:
mean_t=np.transpose(mean)
std_t=np.transpose(std)


Creo DataFrame per mean and std

In [6]:


df_mean = pd.DataFrame(mean_t[1:,1:177],index=mean[0][1:98],columns=region[1:177])
df_std=pd.DataFrame(std_t[1:,1:177],index=std[0][1:98],columns=region[1:177])

df_mean.head()

,Left-Lateral-Ventricle,Left-Inf-Lat-Vent,Left-Cerebellum-White-Matter,Left-Cerebellum-Cortex,Left-Thalamus,Left-Caudate,Left-Putamen,Left-Pallidum,3rd-Ventricle,4th-Ventricle,...,wm-rh-superiorfrontal,wm-rh-superiorparietal,wm-rh-superiortemporal,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-UnsegmentedWhiteMatter,Right-UnsegmentedWhiteMatter
1.0,0.000078,0.002625,0.001393,0.000837,0.001063,0.001086,0.001244,0.000855,0.000820,0.002566,...,0.000842,0.000906,0.000964,0.000797,0.000786,0.000848,0.000919,0.000736,0.000898,0.000931
2.0,0.000093,0.002450,0.001338,0.000724,0.000878,0.000953,0.000986,0.000773,0.000768,0.002419,...,0.000781,0.000774,0.000912,0.000757,0.000752,0.000778,0.000838,0.000749,0.000785,0.000865
3.0,0.000135,0.002526,0.001711,0.000716,0.000955,0.001038,0.001291,0.000905,0.000885,0.002576,...,0.000813,0.000848,0.000803,0.000822,0.000763,0.001048,0.001090,0.000780,0.000910,0.000919
4.0,0.000144,0.002553,0.001665,0.000769,0.001031,0.001050,0.001144,0.000818,0.000747,0.003102,...,0.000830,0.000848,0.000819,0.000839,0.000820,0.000873,0.000873,0.000820,0.000859,0.000903
5.0,0.000142,0.002536,0.001829,0.000781,0.001109,0.000991,0.001134,0.000794,0.000748,0.002780,...,0.000835,0.000823,0.000748,0.000813,0.000765,0.000848,0.000963,0.000766,0.000861,0.000891


Aggiungo i gruppi

In [7]:
a=pd.DataFrame(pd.read_csv('/home/francesco/CompProject/ADNI_dataset_diffusion.csv'))
a.sort_values(by=["Subject"],inplace=True)
group=a["Group"]
a.head()

,Subject,Group,Sex,Age
67,4002,0,F,74.0
68,4003,0,F,72.0
28,4009,1,M,90.0
69,4018,0,M,76.0
70,4050,0,M,77.0


SVM: creo i set e splitto

In [21]:
from sklearn.model_selection import train_test_split


X=df_mean.values
y=group.values

X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, test_size=.2, random_state=6)






MOdello gridsearch


In [27]:
from sklearn import svm

clf = svm.SVC(kernel="rbf")

clf.fit(X_tr, y_tr)
y_pred = clf.predict(X_tst)



[0 1 0 1 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 1]


In [23]:
from sklearn import metrics 


print(metrics.classification_report(y_tst, y_pred)) 

              precision    recall  f1-score   support

           0       0.82      0.69      0.75        13
           1       0.56      0.71      0.63         7

    accuracy                           0.70        20
   macro avg       0.69      0.70      0.69        20
weighted avg       0.73      0.70      0.71        20



In [32]:
from sklearn.model_selection import GridSearchCV
import scipy

r=scipy.stats.expon.rvs(size=100)
g=scipy.stats.expon(scale=.1)
G=g.rvs(size=100)
# defining parameter range 
param_grid = {'C': r, 'gamma': G,
  'kernel': ['linear'], 'class_weight':['balanced', None]}

grid = GridSearchCV(clf, param_grid, refit = True,n_jobs=-1) 




# fitting the model for grid search 
grid.fit(X_tr, y_tr) 
 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_tst) 
print(grid_predictions)
print(y_tst)
# print classification report 
print(metrics.classification_report(y_tst, grid_predictions)) 

set(y_tst) - set(grid_predictions)


{'C': 1.5713977968161452, 'class_weight': 'balanced', 'gamma': 0.03078890901486585, 'kernel': 'linear'}
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.35      1.00      0.52         7

    accuracy                           0.35        20
   macro avg       0.17      0.50      0.26        20
weighted avg       0.12      0.35      0.18        20



/home/francesco/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/francesco/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/francesco/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{0}